<a href="https://colab.research.google.com/github/deepakmoud/Signature-Verification/blob/master/signature_verification_Resnet_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing the Keras libraries and packages
from keras.layers import Input, Flatten, Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg19 import VGG19
from keras.applications.resnet_v2 import ResNet152V2
import warnings 
warnings.filterwarnings('ignore')
img_width, img_height = 128, 128



In [4]:
#Load the pretrained Network
resnet_model=ResNet152V2(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_height,img_width,3), pooling=None, classes=1000)
print("pretrained Network is loaded")


234553344/234545216 [==============================] - 15s 0us/step
pretrained Network is loaded


In [5]:
pip install -U keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [6]:
# Freeze the layers
for layer in resnet_model.layers:
    layer.trainable = False
print("Pretrained layers are freezed")
    

Pretrained layers are freezed


In [7]:
model = Sequential()
model.add(resnet_model)
#add fully connected layer:
input_layer=model.add(Flatten())
hidden_layer=model.add(Dense(128, activation='relu', name='hidden_layer'))
classification_layer=model.add(Dense(64, activation='relu', name='classification_layer'))
output_layer=model.add(Dense(1, activation='sigmoid', name='output_layer')) 
print("All layers top of pretrained layers are developed")

All layers top of pretrained layers are developed


In [8]:
train_data_dir ='/content/drive/My Drive/Dataset1/training_set'
val_data_dir ='/content/drive/My Drive/Dataset1/testing_set'
nb_epochs = 35
print("Input parameters are assigned")

Input parameters are assigned


In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
# Imaga data generation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=False, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=False,class_mode='binary')


Found 2112 images belonging to 2 classes.
Found 544 images belonging to 2 classes.


In [14]:
model_weights_file="/content/drive/My Drive/Colab Notebooks/Signature Verification Resnet Model/model_resnet_weights.h5"
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, nb_epoch=nb_epochs, validation_data=validation_generator)

print('Training Completed!')



Epoch 1/35
66/66 [==============================] - 21s 312ms/step - loss: 0.3617 - accuracy: 0.8438 - val_loss: 1.9848 - val_accuracy: 0.5331
Epoch 2/35
66/66 [==============================] - 20s 300ms/step - loss: 0.4477 - accuracy: 0.7907 - val_loss: 0.7289 - val_accuracy: 0.5000
Epoch 3/35
66/66 [==============================] - 20s 303ms/step - loss: 0.3380 - accuracy: 0.8542 - val_loss: 1.4469 - val_accuracy: 0.5460
Epoch 4/35
66/66 [==============================] - 20s 299ms/step - loss: 0.3961 - accuracy: 0.8234 - val_loss: 0.8917 - val_accuracy: 0.5110
Epoch 5/35
66/66 [==============================] - 20s 301ms/step - loss: 0.3314 - accuracy: 0.8660 - val_loss: 1.7040 - val_accuracy: 0.5312
Epoch 6/35
66/66 [==============================] - 20s 299ms/step - loss: 0.3603 - accuracy: 0.8409 - val_loss: 0.8233 - val_accuracy: 0.4945
Epoch 7/35
66/66 [==============================] - 20s 300ms/step - loss: 0.3191 - accuracy: 0.8646 - val_loss: 0.9900 - val_accuracy: 0.5312

In [15]:
 # save model and architecture to single file
model.save("/content/drive/My Drive/Colab Notebooks/Signature Verification Resnet Model/model_resnet.h5")
model.summary()

print("Saved model to disk")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2 (Model)          (None, 4, 4, 2048)        58331648  
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               4194432   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 62,534,401
Trainable params: 4,202,753
Non-trainable params: 58,331,648
_________________________________________________________________
Saved model to disk


In [16]:
# Loading saved model from Drive.
from keras.models import load_model
model = load_model("/content/drive/My Drive/Colab Notebooks/Signature Verification Resnet Model/model_resnet.h5")
print("Model is Loaded")

Model is Loaded


In [18]:
# Intermediate Model construction
from keras.models import load_model
layer_name= 'classification_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
print("Intermediate model is created")


Intermediate model is created


In [19]:
# Compilation of intermediate model
intermediate_layer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [20]:
# Saving intermediate model
intermediate_layer_model.save("/content/drive/My Drive/Colab Notebooks/Signature Verification Resnet Model/intermediate_model_resnet.h5")
intermediate_layer_model.summary()

print("Saved Intermediate model to disk")

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2_input (InputLaye (None, 128, 128, 3)       0         
_________________________________________________________________
resnet152v2 (Model)          (None, 4, 4, 2048)        58331648  
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               4194432   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 62,534,336
Trainable params: 4,202,688
Non-trainable params: 58,331,648
_________________________________________________________________
Saved Intermediate model to disk


In [21]:
# Loading Intermediate Model
from keras.models import load_model

model = load_model("/content/drive/My Drive/Colab Notebooks/Signature Verification Resnet Model/intermediate_model_resnet.h5")
print("Intermediate model is loaded")

Intermediate model is loaded


In [22]:
# Training Label feature identification(y_train)
 import numpy as np
 batch_size=32
 sample_count=2112
 features = np.zeros(shape=(2112, 64))  # Must be equal to the output of the convolutional base
 labels = np.zeros(shape=(2112))
 i = 0
 for inputs_batch, labels_batch in train_generator:
   features_batch = model.predict(inputs_batch)
   features[i * batch_size: (i + 1) * batch_size] = features_batch
   labels[i * batch_size: (i + 1) * batch_size] = labels_batch
   i += 1
   if i*batch_size  >= sample_count:
     break
 print(labels.shape)

(2112,)


In [24]:
#identification of training Labels
features_train=features
print(features_train.shape)
labels_train=np.expand_dims(labels, axis=1)
print(labels_train.shape)
print(labels_train)#identification of training Labels
features_train=features
print(features_train.shape)
print(features_train)

(2112, 64)
(2112, 1)
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]
(2112, 64)
[[0.         0.         0.         ... 3.71860027 1.58706725 0.        ]
 [0.         0.18674077 0.         ... 0.5415045  0.         0.22154459]
 [0.         0.         0.         ... 0.92697251 0.20752564 0.        ]
 ...
 [0.11829657 1.42003834 3.2898581  ... 5.34149122 0.82854718 4.12055111]
 [0.         1.31627893 0.18501122 ... 2.91357255 0.01922602 2.82411122]
 [0.26623926 0.58577383 1.67538953 ... 2.54005599 0.38738108 1.87591004]]


In [25]:
#identification of training Labels
features_train=features
print(features_train.shape)
labels_train=np.expand_dims(labels, axis=1)
print(labels_train.shape)
print(labels_train)

(2112, 64)
(2112, 1)
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]


In [26]:
# identification of test labels
 import numpy as np
 batch_size=32
 sample_count=544
 features_test = np.zeros(shape=(544, 64))  # Must be equal to the output of the convolutional base
 labels_test = np.zeros(shape=(544))
 i = 0
 for inputs_batch, labels_batch in validation_generator:
   features_batch = model.predict(inputs_batch)
   features_test[i * batch_size: (i + 1) * batch_size] = features_batch
   labels_test[i * batch_size: (i + 1) * batch_size] = labels_batch
   i += 1
   if i*batch_size  >= sample_count:
     break
 print(labels_test.shape)
 print(features_test.shape)
 print(labels_test)

(544,)
(544, 64)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

In [27]:
# #identification of testing Labels
print(features_test.shape)
print(features_test)
labels_test=np.expand_dims(labels_test, axis=1)
print(labels_test.shape)

(544, 64)
[[0.         2.04084635 0.         ... 2.6865294  0.         0.        ]
 [0.67346519 1.39881253 0.02194412 ... 1.62124598 0.         0.        ]
 [0.         0.         0.         ... 1.58509147 0.         0.        ]
 ...
 [0.82288021 1.282938   0.42135772 ... 2.28139424 0.         0.        ]
 [0.48258805 0.         1.07818484 ... 2.71827984 0.         0.        ]
 [1.45386291 0.         1.72805655 ... 4.11900377 0.         0.        ]]
(544, 1)


In [30]:
# SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train,labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[103 165]
 [ 73 203]]
0.5625


In [31]:

# Kernel SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))


[[157 111]
 [106 170]]
0.6011029411764706


In [32]:
# Random Forest
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))



[[169  99]
 [151 125]]
0.5404411764705882


In [33]:
# Decision Tree
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[147 121]
 [149 127]]
0.5036764705882353


In [34]:

# Naive Bayes
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))


[[ 45 223]
 [ 22 254]]
0.5496323529411765


In [35]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier =  KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[140 128]
 [122 154]]
0.5404411764705882


In [36]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[128 140]
 [ 91 185]]
0.5753676470588235


In [37]:
from keras.models import load_model

model = load_model("/content/drive/My Drive/Colab Notebooks/Signature Verification Resnet Model/model_resnet.h5")
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(validation_generator)
print('test loss, test acc:', results)


# Evaluate on test data
17/17 [==============================] - 6s 349ms/step
test loss, test acc: [1.8058701753616333, 0.6084558963775635]
